# Simple Autoencoder for MNIST (with GPU implementation)

### Import necessary libraries

In [1]:
import os #will be used for creating directories, etc.

import torch
import torchvision.datasets as dsets             #for downloading dataset
import torchvision.transforms as transforms      #for transforming dataset into tensors

from torch import nn
import torchvision

from torch.autograd import Variable

from time import time

### Prepare MNIST dataset

In [2]:
#download MNIST dataset
dataset = dsets.MNIST(root='../data', train=True, transform=transforms.ToTensor(), download=True)

### Utility functions

In [3]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

### Training parameters

In [4]:
batch_size = 100
num_epochs = 50

# shuffle and prepare dataset with minibatches
data_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

### Create Autoencoder NN

In [5]:
class Autoencoder(nn.Module):
    def __init__(self, in_dim=784, h_dim=100):
        super(Autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, h_dim),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(h_dim, in_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        out = self.encoder(x)
        out = self.decoder(out)
        return out

In [6]:
hidden_size = 30     #size of the bottleneck layer

# Create Autoencoder model
ae = Autoencoder(in_dim=784, h_dim=hidden_size)

if torch.cuda.is_available():
    ae.cuda()

### Optimizer and loss function

In [7]:
criterion = nn.BCELoss()     #Binary cross-entropy loss for each pixel
optimizer = torch.optim.Adam(ae.parameters(), lr=0.001)

### Let's train!!

In [8]:
# We want to go through all the examples in each epoch, so number of iterations per epoch
iter_per_epoch = len(data_loader)
data_iter      = iter(data_loader) # data_iter is an iterator object

#### Now we want to view and save results on a fixed batch to visualize Autoencoder's performance

In [9]:
# save fixed inputs for debugging
fixed_x, labels = next(data_iter)        #points to the first batch
torchvision.utils.save_image(Variable(fixed_x).data.cpu(), './data/real_images.png')
print(labels.size())

fixed_x = to_var(fixed_x.view(fixed_x.size(0), -1))
print(fixed_x.size())

torch.Size([100])
torch.Size([100, 784])


<built-in method type of Tensor object at 0x0000029C055CDEE8>


### Begin training

In [10]:
for epoch in range(num_epochs):
    t0 = time()
    for i, (images, _) in enumerate(data_loader):
        
        # flatten the image
        images = to_var(images.view(images.size(0), -1))
        out = ae(images)
        loss = criterion(out, images)
        
        optimizer.zero_grad()
        loss.backward()         # calculate gradients
        optimizer.step()        # update parameters
        
        # display training process after every 100 iterations
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f Time: %.2fs' 
                %(epoch+1, num_epochs, i+1, len(dataset)//batch_size, loss.data[0], time()-t0))
            
    # save the reconstructed images
    reconst_images = ae(fixed_x)
    reconst_images = reconst_images.view(reconst_images.size(0), 1, 28, 28)
    torchvision.utils.save_image(reconst_images.data.cpu(), './data/reconst_images_%d.png' % (epoch+1))

C:\Users\mayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch [1/50], Iter [100/600] Loss: 0.2738 Time: 2.25s
Epoch [1/50], Iter [200/600] Loss: 0.2343 Time: 3.28s
Epoch [1/50], Iter [300/600] Loss: 0.2123 Time: 4.34s
Epoch [1/50], Iter [400/600] Loss: 0.1892 Time: 5.35s
Epoch [1/50], Iter [500/600] Loss: 0.1906 Time: 6.36s
Epoch [1/50], Iter [600/600] Loss: 0.1729 Time: 7.37s
Epoch [2/50], Iter [100/600] Loss: 0.1698 Time: 1.06s
Epoch [2/50], Iter [200/600] Loss: 0.1706 Time: 2.09s
Epoch [2/50], Iter [300/600] Loss: 0.1453 Time: 3.12s
Epoch [2/50], Iter [400/600] Loss: 0.1618 Time: 4.19s
Epoch [2/50], Iter [500/600] Loss: 0.1427 Time: 5.26s
Epoch [2/50], Iter [600/600] Loss: 0.1412 Time: 6.33s
Epoch [3/50], Iter [100/600] Loss: 0.1445 Time: 1.08s
Epoch [3/50], Iter [200/600] Loss: 0.1522 Time: 2.18s
Epoch [3/50], Iter [300/600] Loss: 0.1303 Time: 3.23s
Epoch [3/50], Iter [400/600] Loss: 0.1376 Time: 4.28s
Epoch [3/50], Iter [500/600] Loss: 0.1373 Time: 5.35s
Epoch [3/50], Iter [600/600] Loss: 0.1367 Time: 6.37s
Epoch [4/50], Iter [100/600]

Epoch [50/50], Iter [400/600] Loss: 0.1249 Time: 4.13s
Epoch [50/50], Iter [500/600] Loss: 0.1234 Time: 5.18s
Epoch [50/50], Iter [600/600] Loss: 0.1203 Time: 6.19s


In [ ]:
for i, (images, _) in enumerate(data_loader):